In [1]:
# VYTVORENIE DATASETU observed.csv

In [1]:
# BLOK 1
# Importovanie kniznic.

import os, json
import pandas as pd
import numpy as np

In [2]:
# BLOK 2
# Nacitanie suborov. Vytvorenie prazdnej tabulky.

path_to_json = 'krivky'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
df=pd.DataFrame(columns=["Q", "Inc", "Omega1", "Omega2", "T1", "T2", "P", "Type", "Spot", "Ref", "Name", "Filter", "Curve"])

In [3]:
# BLOK 3
# Nacitanie dat do tabulky. Zoradenie dat v tabulke.

p=0
for index, js in enumerate(json_files):
    with open(os.path.join(path_to_json, js)) as json_file:
        json_text = json.load(json_file)
        n=len(json_text["interpolated_light_curves"])
        f=np.array(list(json_text["interpolated_light_curves"].keys()))
        for i in range(n-1):
            q = json_text["system_parameters"]["q"]
            inc = json_text["system_parameters"]["inc"]
            omega1 = json_text["system_parameters"]["omega1"]
            omega2 = json_text["system_parameters"]["omega2"]
            T1 = json_text["system_parameters"]["T1"]
            T2 = json_text["system_parameters"]["T2"]
            P = json_text["system_parameters"]["P"]
            Type = json_text["system_parameters"]["Type"]
            Spot = json_text["system_parameters"]["Spot"]
            Ref = json_text["system_parameters"]["Ref"]
            name = js[:-5]
            Filter = f[i+1]
            curve = json_text["interpolated_light_curves"][Filter]
            df.loc[p] = [q, inc, omega1, omega2, max(T1, T2), min(T1, T2), P, Type, Spot, Ref, name, Filter, curve]
            p=p+1
            
df["Type"]=df["Type"].map({"contact":"overcontact", "overcontact":"overcontact", "detached":"detached", "detected":"detached"})            
df=df.sort_values(by=["Type", "Name"])

In [4]:
# Vypisanie statistickych informacii o oddelenych systemoch

df.loc[df.Type.isin(['detached'])].describe()

,Q,Inc,Omega1,Omega2,T1,T2,P
count,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000
mean,0.416475,83.170500,4.359175,3.070900,9534.500000,6418.675000,1.868477
std,0.268703,3.803218,1.620190,1.666744,5584.027106,5078.932178,1.386343
min,0.135000,76.300000,2.540000,2.062000,5900.000000,3450.000000,0.406916
25%,0.184000,79.900000,3.181750,2.190000,7400.000000,4478.000000,0.796455
50%,0.352000,83.600000,4.230000,2.580000,8000.000000,4893.000000,1.548869
75%,0.484000,85.800000,4.550000,2.850000,8770.000000,5707.000000,2.587332
max,1.000000,88.700000,8.550000,8.550000,28000.000000,23640.000000,5.520188


In [5]:
# Vypisanie poctu kriviek pre dany filter pre oddelene systemy
print('----------detached----------')
for key, value in dict(df.loc[df.Type.isin(['detached'])]['Filter'].value_counts()).items():
    print('Filter: ' + key + '     count: ' + str(value))

----------detached----------
Filter: V     count: 14
Filter: B     count: 12
Filter: R     count: 10
Filter: I     count: 4


In [6]:
# Vypisanie statistickych informacii o dotykovych systemoch


df.loc[df.Type.isin(['overcontact'])].describe()

,Q,Inc,Omega1,Omega2,T1,T2,P
count,46.000000,46.000000,46.000000,46.000000,46.000000,46.000000,46.000000
mean,0.642152,76.751087,2.945478,2.945087,6191.913043,5857.413043,0.408642
std,0.847369,9.379962,1.309041,1.309178,639.928914,677.966947,0.101349
min,0.169000,54.740000,2.080000,2.080000,4908.000000,4634.000000,0.234728
25%,0.250000,70.900000,2.317000,2.317000,5776.500000,5536.000000,0.375307
50%,0.290000,80.050000,2.396000,2.396000,6280.000000,6141.000000,0.405892
75%,0.470000,83.300000,2.783000,2.783000,6500.000000,6250.000000,0.456274
max,2.932000,86.200000,6.397000,6.397000,7000.000000,6750.000000,0.597423


In [7]:
# Vypisanie poctu kriviek pre dany filter pre dotykove systemy

print('----------overcontact----------')
for key, value in dict(df.loc[df.Type.isin(['overcontact'])]['Filter'].value_counts()).items():
    print('Filter: ' + key + '     count: ' + str(value))

----------overcontact----------
Filter: V     count: 15
Filter: B     count: 12
Filter: R     count: 12
Filter: I     count: 7


In [26]:
# BLOK 4
# Ulozenie dat v podobe tabulky do suboru .csv.

df.to_csv("observed.csv")